In [1]:
import pandas as pd

from os import listdir, path
from os.path import isfile, join

from matplotlib import pyplot as plt
from scipy.interpolate import interp1d

# from tabulate import tabulate
import numpy as np


from mpl_toolkits.axes_grid1.inset_locator import inset_axes 
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [2]:
# dataset_list = ['mnist','blog','income','cifar10']
dataset_list = ['aloi','blog','adult','syn','sensorless','covtype','helena','higgs_small']
# dataset_list = ['syn','cifar10']

In [3]:
allFiles=[]
for dataset in dataset_list :
    mypath = "results/"+ dataset +"/"
    onlyfiles = [mypath + f for f in listdir(mypath) if isfile(join(mypath, f))]
    onlyfiles = [f for f in onlyfiles if f[-3:] == "csv"]
    allFiles.extend(onlyfiles)
# allFiles


In [4]:
allFiles[0]

'results/aloi/Client-1-contrastive-50e-6fl-0.5mc-scale_at-0.5rl-aloi.csv'

In [5]:
for i, item in enumerate(allFiles):
    dt = pd.read_csv(item)
    dt["client"] = int(item.split("-")[1])
    # dt["type"] = item.split("-")[-1].split(".")[0]
    dt["epoch"] = int(item.split("-")[3][:-1])
    dt["fl"] = int(item.split("-")[4][:-2])
    dt["mc"] = float(item.split("-")[5][:-2])
    dt["at"] = str(item.split("-")[6][:-3])
    dt["rl"] = float(item.split("-")[7][:-2])
    dt["db"] = str(item.split("-")[8][:-4])
    dt["f1"] = dt.test_acc.apply(lambda x : eval(x)[-2])
    dt["accuracy"] = dt.test_acc.apply(lambda x : eval(x)[0])
    
    if i ==0 : db = dt
    else : db = pd.concat([db,dt])
# db = pd.merge(db[db.type == 'original'], db[db.type == 'contrastive'], on=["client","model"])
db = db[['client',	'epoch',	'fl',	'mc',	'at',	'rl',	'db','accuracy','f1']]
db = db[db.epoch == 50]
# db.head()

In [6]:
db.db.unique()

array(['aloi', 'blog', 'adult', 'syn', 'sensorless', 'covtype', 'helena',
       'higgs_small'], dtype=object)

In [7]:
db['combined'] = db[[  'mc', 'rl']].apply(
    lambda x: '_'.join(x.astype(str)), axis=1
)
idx = [x+ '_' +item+'_'+str(n) for n in range(6) for item in db['at'].unique() for x in db.sort_values(['mc','rl'])['combined'].unique()]
idx = pd.DataFrame(idx, columns=['idx'])
idx.shape

(312, 1)

In [8]:
db['combined'] = db[[  'mc', 'rl','at','client']].apply(
    lambda x: '_'.join(x.astype(str)), axis=1
)
# db['combined']

## Explore

In [12]:
def printDb(dataset):
# dataset = 'aloi'
    print('Dataset used:', dataset)
    dbAloi = db[db['db'] == dataset]
    dbx = idx.merge(dbAloi,left_on='idx', 
                          right_on='combined',
                          how='left')
    dbx = dbx[['idx','client', 'mc', 'at', 'rl', 'db', 'accuracy',
           'f1',]]
    # dbx.isna()
    # dbx.idx.apply(lambda x : x.split('_'))
    dbx.client = dbx.idx.apply(lambda x : x.split('_')[-1])
    dbx.mc = dbx.idx.apply(lambda x : x.split('_')[0])
    dbx.rl = dbx.idx.apply(lambda x : x.split('_')[1])
    dbx['at'] = dbx.idx.apply(lambda x : x.split('_')[-2])
    dbx['nl'] = dbx.f1.isna()
    dbx['nnl'] = dbx.f1.notna()
    
    dbx['db'] = dataset
    dbx = dbx.drop(
        dbx.query("mc == '0.0' and at!= 'scale'").index
    )
    # dbx[['mc','rl','at','f1','accuracy']].groupby(['mc','rl','at'])[['f1','accuracy']].apply(lambda x: x.isna().sum())
    dbNl = dbx[['mc','rl','at','nl']].pivot_table(
                    values='nl', 
                    index=['mc','rl',],
                    columns='at',
                    aggfunc='sum').reset_index().astype('float')
    print('***Fail Matrix***')
    print(dbNl)
    print(dbNl[['mc','replacement','scale','targeted']].groupby('mc').sum()/24)
    print(dbNl[['rl','replacement','scale','targeted']].groupby('rl').sum() / 24)
    # dbNnl = 
    dbx.f1 = dbx.f1.apply(lambda x : x if x>0 else np.nan)
    dbF1 = dbx[['mc','rl','at','f1']].pivot_table(
                    values='f1', 
                    index=['mc','rl',],
                    columns='at',
                    aggfunc='mean').reset_index().astype('float')
    print('***F1 Matrix***')
    print(dbF1)
    print(dbF1[['mc','replacement','scale','targeted']].groupby('mc').mean())
    print(dbF1[['rl','replacement','scale','targeted']].groupby('rl').mean())
    print('***End of Matrixs***')


printDb('helena')

Dataset used: helena
***Fail Matrix***
at    mc    rl  ascent  replacement  scale  targeted
0   0.00  1.00     NaN          NaN    0.0       NaN
1   0.25  0.25     0.0          0.0    0.0       0.0
2   0.25  0.50     0.0          0.0    0.0       0.0
3   0.25  0.75     0.0          0.0    0.0       0.0
4   0.25  1.00     0.0          0.0    0.0       0.0
5   0.50  0.25     6.0          0.0    0.0       0.0
6   0.50  0.50     6.0          0.0    0.0       0.0
7   0.50  0.75     6.0          0.0    0.0       0.0
8   0.50  1.00     6.0          0.0    0.0       0.0
9   0.75  0.25     6.0          0.0    0.0       0.0
10  0.75  0.50     6.0          0.0    0.0       0.0
11  0.75  0.75     6.0          0.0    0.0       0.0
12  0.75  1.00     6.0          0.0    0.0       0.0
at    replacement  scale  targeted
mc                                
0.00          0.0    0.0       0.0
0.25          0.0    0.0       0.0
0.50          0.0    0.0       0.0
0.75          0.0    0.0       0.0
at    rep

In [11]:
# for item in dataset_list:
#     printDb(item)

In [ ]:
# def printDb(dataset): 
dataset = 'aloi'
print('Dataset used:', dataset)
dbAloi = db[db['db'] == dataset]
dbx = idx.merge(dbAloi,left_on='idx', 
                      right_on='combined',
                      how='left')
dbx = dbx[['idx','client', 'mc', 'at', 'rl', 'db', 'accuracy',
       'f1',]]
# dbx.isna()
# dbx.idx.apply(lambda x : x.split('_'))
dbx.client = dbx.idx.apply(lambda x : x.split('_')[-1])
dbx.mc = dbx.idx.apply(lambda x : x.split('_')[0])
dbx.rl = dbx.idx.apply(lambda x : x.split('_')[1])
dbx['at'] = dbx.idx.apply(lambda x : x.split('_')[-2])
dbx['nl'] = dbx.f1.isna()
dbx['nnl'] = dbx.f1.notna()

dbx['db'] = dataset
dbx = dbx.drop(
    dbx.query("mc == '0.0' and at!= 'scale'").index
)
# dbx[['mc','rl','at','f1','accuracy']].groupby(['mc','rl','at'])[['f1','accuracy']].apply(lambda x: x.isna().sum())
dbNl = dbx[['mc','rl','at','nl']].pivot_table(
                values='nl', 
                index=['mc','rl',],
                columns='at',
                aggfunc='sum').reset_index().astype('float')
print('***Fail Matrix***')
print(dbNl)
print(dbNl[['mc','replacement','scale','targeted']].groupby('mc').sum()/24)
print(dbNl[['rl','replacement','scale','targeted']].groupby('rl').sum() / 24)
# dbNnl = 
dbx.f1 = dbx.f1.apply(lambda x : x if x>0 else np.nan)
dbF1 = dbx[['mc','rl','at','f1']].pivot_table(
                values='f1', 
                index=['mc','rl',],
                columns='at',
                aggfunc='mean').reset_index().astype('float')
print('***F1 Matrix***')
print(dbF1)
print(dbF1[['mc','replacement','scale','targeted']].groupby('mc').mean())
print(dbF1[['rl','replacement','scale','targeted']].groupby('rl').mean())
print('***End of Matrixs***')




In [214]:
# dx = db[(db.client_drop>0)&(db.class_imbalance>0)]\
# [['model','test_acc', 'client', 'type', 'dataset']]\
# .pivot(index=['dataset','model','client'],
#        values=['test_acc'],
#        columns=['type'])\
# .reset_index()
# dx.columns = ['dataset','model','client','fl','SubTab','local']
# dx.set_index(['client','dataset'], inplace=True)
# dx = dx[dx.model=='CFL'][['fl','local']].join(
#     dx[dx.model=='LL'][['local']], rsuffix='_LL').join(
#     dx[dx.model=='SubTab'][['SubTab']], rsuffix='_LL').join(
#     dx[dx.model=='SubTab FL'][['SubTab']], rsuffix='_LL').reset_index()
# dx.columns = ['client', 'dataset', 'CFL', 'C','LL','SubTab', 'SubtTab FL']
# dx = dx[['client', 'dataset', 'CFL', 'C','LL','SubTab', 'SubtTab FL']]

# (dx.groupby('dataset')[['CFL']].mean().style.highlight_min(color = 'red', axis = 1))\
# .highlight_max(color = 'lightgreen', axis = 1)